In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

DATA_PATH = 'Data'
GESTURES = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
NUM_LANDMARKS = 63

X = []
y = []

for gesture in GESTURES:
    gesture_path = os.path.join(DATA_PATH, gesture)
    for file in os.listdir(gesture_path):
        file_path = os.path.join(gesture_path, file)
        landmarks = np.load(file_path)
        
        X.append(landmarks)
        y.append(gesture)

X = np.array(X)
y = np.array(y)

X = X / np.max(X)

lb = LabelBinarizer()
y = lb.fit_transform(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reshape(X_train.shape[0], NUM_LANDMARKS, 1)
X_val = X_val.reshape(X_val.shape[0], NUM_LANDMARKS, 1)

print(f'Training data shape: {X_train.shape}')
print(f'Validation data shape: {X_val.shape}')


Training data shape: (1621, 63, 1)
Validation data shape: (406, 63, 1)


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(NUM_LANDMARKS, 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 61, 64)            256       
                                                                 
 max_pooling1d (MaxPooling1  (None, 30, 64)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 28, 128)           24704     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 14, 128)           0         
 g1D)                                                            
                                                                 
 flatten (Flatten)           (None, 1792)              0         
                                                                 
 dense (Dense)               (None, 128)               2

In [3]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,  
    batch_size=32,
    verbose=2
)

model.save('model_numbers.h5')


Epoch 1/30
51/51 - 3s - loss: 2.2664 - accuracy: 0.1357 - val_loss: 2.1499 - val_accuracy: 0.2291 - 3s/epoch - 56ms/step
Epoch 2/30
51/51 - 0s - loss: 1.8739 - accuracy: 0.2517 - val_loss: 1.5054 - val_accuracy: 0.3867 - 482ms/epoch - 9ms/step
Epoch 3/30
51/51 - 0s - loss: 1.4743 - accuracy: 0.3751 - val_loss: 1.1578 - val_accuracy: 0.6429 - 466ms/epoch - 9ms/step
Epoch 4/30
51/51 - 0s - loss: 1.1598 - accuracy: 0.5238 - val_loss: 0.8443 - val_accuracy: 0.7291 - 473ms/epoch - 9ms/step
Epoch 5/30
51/51 - 0s - loss: 0.9678 - accuracy: 0.5935 - val_loss: 0.6124 - val_accuracy: 0.7734 - 459ms/epoch - 9ms/step
Epoch 6/30
51/51 - 0s - loss: 0.8219 - accuracy: 0.6317 - val_loss: 0.5090 - val_accuracy: 0.8793 - 481ms/epoch - 9ms/step
Epoch 7/30
51/51 - 0s - loss: 0.7082 - accuracy: 0.6909 - val_loss: 0.4789 - val_accuracy: 0.7783 - 449ms/epoch - 9ms/step
Epoch 8/30
51/51 - 0s - loss: 0.6257 - accuracy: 0.7353 - val_loss: 0.3335 - val_accuracy: 0.9187 - 448ms/epoch - 9ms/step
Epoch 9/30
51/51 -

c:\Users\LENOVO\anaconda3\envs\myenv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {val_acc}')


13/13 [==============================] - 0s 5ms/step - loss: 0.0138 - accuracy: 0.9975
Validation accuracy: 0.9975369572639465
